In [9]:
# --- 1. Mount Google Drive and Set Project Directory ---
import os
import sys
import subprocess

PROJECT_DIR = "/content/drive/MyDrive/ROME/rome-main"

if 'google.colab' in sys.modules:
    from google.colab import drive
    if not os.path.exists('/content/drive/MyDrive'):
        print("Mounting Google Drive...")
        drive.mount('/content/drive')
    else:
        print("Google Drive already mounted.")

if not os.path.exists(PROJECT_DIR):
    print(f"Error: Project directory not found at {PROJECT_DIR}")
    print("Stopping script. Please verify the path.")
    # Stop execution if the path is wrong
    sys.exit("Project directory not found.")
else:
    os.chdir(PROJECT_DIR)
    print(f"Successfully changed working directory to: {os.getcwd()}")

# --- 2. Install Dependencies ---
print("Installing dependencies...")
!pip install transformers datasets tokenizers python-dotenv hydra-core higher einops

import nltk
print("Downloading NLTK data...")
nltk.download('punkt')
# Note: 'punkt_tab' is not a standard NLTK package. 'punkt' is correct.
# nltk.download('punkt_tab') # This line from your code is likely an error and not needed.

print("\n--- Setup Complete ---")

Mounting Google Drive...
Mounted at /content/drive
Successfully changed working directory to: /content/drive/MyDrive/ROME/rome-main
Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



--- Setup Complete ---


In [6]:
# --- 3. Run All Evaluations for Table 1 (zsRE) ---
# This will take a very long time, especially on gpt2-xl.
# Each algorithm runs sequentially. The results are saved automatically.

MODEL_NAME = "gpt2-xl"
DS_NAME = "zsre"

print(f"--- Running ROME for {DS_NAME} on {MODEL_NAME} ---")
!python3 -m experiments.evaluate \
    --alg_name=ROME \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}.json \
    --ds_name={DS_NAME} \
    --conserve_memory

print(f"\n--- Running FT (Fine-Tuning) for {DS_NAME} on {MODEL_NAME} ---")
# Note: The paper lists "FT" and "FT+L". We'll run the constrained version.
# You may need to investigate hparams/FT/gpt2-xl_unconstr.json for the other.
!python3 -m experiments.evaluate \
    --alg_name=FT \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}_constr.json \
    --ds_name={DS_NAME} \
    --conserve_memory

print(f"\n--- Running KE (KnowledgeEditor) for {DS_NAME} on {MODEL_NAME} ---")
# We use the specific zsRE hparams file
!python3 -m experiments.evaluate \
    --alg_name=KE \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}_zsRE.json \
    --ds_name={DS_NAME} \
    --conserve_memory

print(f"\n--- Running MEND for {DS_NAME} on {MODEL_NAME} ---")
# We use the specific zsRE hparams file
!python3 -m experiments.evaluate \
    --alg_name=MEND \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}_zsRE.json \
    --ds_name={DS_NAME} \
    --conserve_memory

print("\n--- All Evaluations Complete ---")

Streaming output truncated to the last 5000 lines.
Epoch: 1
Batch loss 4.026057243347168
Total loss 4.026057243347168
Epoch: 2
Batch loss 3.482276439666748
Total loss 3.482276439666748
Epoch: 3
Batch loss 2.816505193710327
Total loss 2.816505193710327
Epoch: 4
Batch loss 2.2526891231536865
Total loss 2.2526891231536865
Epoch: 5
Batch loss 1.839945673942566
Total loss 1.839945673942566
Epoch: 6
Batch loss 1.4952294826507568
Total loss 1.4952294826507568
Epoch: 7
Batch loss 1.1431949138641357
Total loss 1.1431949138641357
Epoch: 8
Batch loss 0.8780083060264587
Total loss 0.8780083060264587
Epoch: 9
Batch loss 0.635308027267456
Total loss 0.635308027267456
Epoch: 10
Batch loss 0.432330459356308
Total loss 0.432330459356308
Epoch: 11
Batch loss 0.3058132529258728
Total loss 0.3058132529258728
Epoch: 12
Batch loss 0.23767247796058655
Total loss 0.23767247796058655
Epoch: 13
Batch loss 0.18554210662841797
Total loss 0.18554210662841797
Epoch: 14
Batch loss 0.1503084897994995
Total loss 0.150

In [ ]:
# --- 3. Run All Evaluations for Table 1 (zsRE) ---
# [MODIFIED FOR SPEED]
# We are using --dataset_size_limit=100 to run on a small subset.
# This is great for a quick test, but the results will not be
# statistically significant or match the paper's table.
# For the final run, remove this line or set it to a large number.

MODEL_NAME = "gpt2-xl"
DS_NAME = "zsre"
SAMPLE_SIZE = 20  # Set your desired sample size here

print(f"--- Running ROME for {DS_NAME} on {MODEL_NAME} (SAMPLE_SIZE={SAMPLE_SIZE}) ---")
!python3 -m experiments.evaluate \
    --alg_name=ROME \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}.json \
    --ds_name={DS_NAME} \
    --dataset_size_limit={SAMPLE_SIZE} \
    --conserve_memory

print(f"\n--- Running FT (Fine-Tuning) for {DS_NAME} on {MODEL_NAME} (SAMPLE_SIZE={SAMPLE_SIZE}) ---")
!python3 -m experiments.evaluate \
    --alg_name=FT \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}_constr.json \
    --ds_name={DS_NAME} \
    --dataset_size_limit={SAMPLE_SIZE} \
    --conserve_memory

print(f"\n--- Running KE (KnowledgeEditor) for {DS_NAME} on {MODEL_NAME} (SAMPLE_SIZE={SAMPLE_SIZE}) ---")
!python3 -m experiments.evaluate \
    --alg_name=KE \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}_zsRE.json \
    --ds_name={DS_NAME} \
    --dataset_size_limit={SAMPLE_SIZE} \
    --conserve_memory

print(f"\n--- Running MEND for {DS_NAME} on {MODEL_NAME} (SAMPLE_SIZE={SAMPLE_SIZE}) ---")
!python3 -m experiments.evaluate \
    --alg_name=MEND \
    --model_name={MODEL_NAME} \
    --hparams_fname={MODEL_NAME}_zsRE.json \
    --ds_name={DS_NAME} \
    --dataset_size_limit={SAMPLE_SIZE} \
    --conserve_memory

print("\n--- All Sample Evaluations Complete ---")

Streaming output truncated to the last 5000 lines.
Batch loss 5.222438812255859
Total loss 5.222438812255859
Epoch: 11
Batch loss 4.8818678855896
Total loss 4.8818678855896
Epoch: 12
Batch loss 4.431390285491943
Total loss 4.431390285491943
Epoch: 13
Batch loss 3.88623046875
Total loss 3.88623046875
Epoch: 14
Batch loss 3.478832721710205
Total loss 3.478832721710205
Epoch: 15
Batch loss 3.068352699279785
Total loss 3.068352699279785
Epoch: 16
Batch loss 2.643547534942627
Total loss 2.643547534942627
Epoch: 17
Batch loss 2.303978681564331
Total loss 2.303978681564331
Epoch: 18
Batch loss 2.028416633605957
Total loss 2.028416633605957
Epoch: 19
Batch loss 1.7376606464385986
Total loss 1.7376606464385986
Epoch: 20
Batch loss 1.6076637506484985
Total loss 1.6076637506484985
Epoch: 21
Batch loss 1.5824918746948242
Total loss 1.5824918746948242
Epoch: 22
Batch loss 2.2619078159332275
Total loss 2.2619078159332275
Epoch: 23
Batch loss 1.5931633710861206
Total loss 1.5931633710861206
Epoch: 24

In [4]:
print("--- Summarizing ROME Results ---")
!python3 -m experiments.summarize --dir_name=ROME --runs=run_001

--- Summarizing ROME Results ---
{'num_cases': 52,
 'post_neighborhood_acc': (np.float64(28.06), np.float64(27.05)),
 'post_paraphrase_acc': (np.float64(79.19), np.float64(31.43)),
 'post_rewrite_acc': (np.float64(99.45), np.float64(3.92)),
 'post_score': (np.float64(51.44), np.float64(nan)),
 'pre_neighborhood_acc': (np.float64(28.1), np.float64(27.06)),
 'pre_paraphrase_acc': (np.float64(20.74), np.float64(25.16)),
 'pre_rewrite_acc': (np.float64(20.74), np.float64(25.39)),
 'pre_score': (np.float64(22.72), np.float64(nan)),
 'run_dir': 'results/ROME/run_001',
 'time': (np.float64(4.263914351279919), np.float64(1.9926132635145974))}


In [10]:
# --- 4. Summarize and Report Results ---
# This script reads the files saved in step 3 and prints the metrics.
# We assume the run ID for each is 'run_000'.
# If you run this multiple times, you may need to update 'run_000' to 'run_001', etc.

print("--- Summarizing ROME Results ---")
!python3 -m experiments.summarize --dir_name=ROME --runs=run_000

print("\n--- Summarizing FT Results ---")
!python3 -m experiments.summarize --dir_name=FT --runs=run_000

print("\n--- Summarizing KE Results ---")
!python3 -m experiments.summarize --dir_name=KE --runs=run_000

print("\n--- Summarizing MEND Results ---")
!python3 -m experiments.summarize --dir_name=MEND --runs=run_000

--- Summarizing ROME Results ---

--- Summarizing FT Results ---
{'num_cases': 722,
 'post_neighborhood_acc': (np.float64(23.3), np.float64(23.78)),
 'post_paraphrase_acc': (np.float64(32.97), np.float64(33.54)),
 'post_rewrite_acc': (np.float64(61.91), np.float64(36.26)),
 'post_score': (np.float64(33.56), np.float64(nan)),
 'pre_neighborhood_acc': (np.float64(24.47), np.float64(24.01)),
 'pre_paraphrase_acc': (np.float64(24.16), np.float64(27.54)),
 'pre_rewrite_acc': (np.float64(24.8), np.float64(28.01)),
 'pre_score': (np.float64(24.47), np.float64(nan)),
 'run_dir': 'results/FT/run_000',
 'time': (np.float64(2.0353764989039242), np.float64(0.22234567675231554))}

--- Summarizing KE Results ---
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/drive/MyDrive/ROME/rome-main/experiments/summarize.py", line 180, in <module>
    main(
  File "/content/drive/MyDrive/ROME/rome-main/e

In [4]:
# --- 5. Format as Table 1 ---
# After running the summarize scripts above, manually copy the
# 'Efficacy' (score_post), 'Paraphrase' (para_post), and 'Specificity' (spec_post)
# values printed to the console into the data dictionary below.

import pandas as pd

# Manually update this dictionary with the output from Step 4.
# The values here are placeholders based on the paper's table.
# You will need to replace "efficacy_val", "paraphrase_val", etc.
# with the numbers you get from the console output.

# The summarize script will output metrics like:
# {
#   ...
#   "score_post": [99.8, 0.0],  <-- This is Efficacy
#   "para_post": [88.1, 0.5],   <-- This is Paraphrase
#   "spec_post": [24.2, 0.5]    <-- This is Specificity
#   ...
# }
# You want the first number (the mean).

data = {
    "Editor": ["ROME", "FT", "KE", "MEND"],
    "Efficacy": [
        "99.8 (±0.0)",  # Replace with your ROME score_post
        "99.6 (±0.1)",  # Replace with your FT score_post
        "92.4 (±0.3)",  # Replace with your KE score_post
        "75.9 (±0.5)"   # Replace with your MEND score_post
    ],
    "Paraphrase": [
        "88.1 (±0.5)",  # Replace with your ROME para_post
        "92.3 (±0.4)",  # Replace with your FT para_post
        "47.2 (±0.7)",  # Replace with your KE para_post
        "23.8 (±0.5)"   # Replace with your MEND para_post
    ],
    "Specificity": [
        "24.2 (±0.5)",  # Replace with your ROME spec_post
        "23.2 (±0.5)",  # Replace with your FT spec_post
        "23.4 (±0.5)",  # Replace with your KE spec_post
        "24.1 (±0.5)"   # Replace with your MEND spec_post
    ]
}

df = pd.DataFrame(data)
df.set_index("Editor", inplace=True)

print("--- Reproduced Table 1: zsRE Editing Results on GPT-2 XL ---")
print(df.to_markdown(numalign="center", stralign="center"))

2025-11-05 05:37:01.898615: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 05:37:01.915451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762321021.936295    2433 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762321021.942649    2433 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762321021.958729    2433 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
import numpy as np
from scipy.stats import hmean
from typing import Dict, Tuple

# --- Metric Definitions and Calculation Functions ---

def calculate_efficacy(
    P_star_rewrite: np.ndarray, P_orig_rewrite: np.ndarray
) -> Tuple[float, float, float, float]:
    """
    Calculates Efficacy Score (ES) and Efficacy Magnitude (EM).

    ES: Portion of cases where P[o*] > P[o^c]. (Goal: High)
    EM: Mean difference P[o*] - P[o^c]. (Goal: High)
    """
    # Efficacy Magnitude (EM)
    EM_diff = P_star_rewrite - P_orig_rewrite
    EM = np.mean(EM_diff)
    EM_std = np.std(EM_diff)

    # Efficacy Score (ES)
    ES_success = (P_star_rewrite > P_orig_rewrite).astype(float)
    ES = np.mean(ES_success) * 100.0  # Convert to percentage
    ES_std = np.std(ES_success) * 100.0

    return ES, ES_std, EM, EM_std

def calculate_generalization(
    P_star_paraphrase: np.ndarray, P_orig_paraphrase: np.ndarray
) -> Tuple[float, float, float, float]:
    """
    Calculates Paraphrase Score (PS) and Paraphrase Magnitude (PM).
    (Measures generalization to rephrased prompts.)

    PS: Portion of cases where P[o*] > P[o^c] on paraphrases. (Goal: High)
    PM: Mean difference P[o*] - P[o^c] on paraphrases. (Goal: High)
    """
    # Paraphrase Magnitude (PM)
    PM_diff = P_star_paraphrase - P_orig_paraphrase
    PM = np.mean(PM_diff)
    PM_std = np.std(PM_diff)

    # Paraphrase Score (PS)
    PS_success = (P_star_paraphrase > P_orig_paraphrase).astype(float)
    PS = np.mean(PS_success) * 100.0
    PS_std = np.std(PS_success) * 100.0

    return PS, PS_std, PM, PM_std

def calculate_specificity(
    P_star_neighbor: np.ndarray, P_orig_neighbor: np.ndarray
) -> Tuple[float, float, float, float]:
    """
    Calculates Neighborhood Score (NS) and Neighborhood Magnitude (NM).
    (Measures specificity/localization, i.e., preserving original facts
    for neighboring subjects.)

    NS: Portion of cases where P[o^c] > P[o*] on neighbor prompts. (Goal: High)
    NM: Mean difference P[o^c] - P[o*] on neighbor prompts. (Goal: High)
    """
    # Neighborhood Magnitude (NM)
    # Note: P[o^c] - P[o*] is used for NM to match the paper's goal of positive value
    # for preserved original fact.
    NM_diff = P_orig_neighbor - P_star_neighbor
    NM = np.mean(NM_diff)
    NM_std = np.std(NM_diff)

    # Neighborhood Score (NS)
    NS_success = (P_orig_neighbor > P_star_neighbor).astype(float)
    NS = np.mean(NS_success) * 100.0
    NS_std = np.std(NS_success) * 100.0

    return NS, NS_std, NM, NM_std

def calculate_score_s(ES: float, PS: float, NS: float) -> float:
    """
    Calculates the overall Score (S) as the harmonic mean of ES, PS, and NS.
    S is reported as a percentage (0-100).
    """
    # Harmonic mean requires fractions, so divide by 100
    scores_frac = np.array([ES / 100.0, PS / 100.0, NS / 100.0])
    # The harmonic mean is only defined for positive numbers,
    # so we exclude any zero scores (though ideal scores are > 0)
    positive_scores = scores_frac[scores_frac > 0]

    if len(positive_scores) == 0:
        return 0.0

    # hmean returns a fraction; multiply by 100 for percentage
    S = hmean(positive_scores) * 100.0
    return S

def process_results(data: Dict) -> Dict:
    """Processes raw data dictionary into the final Table 4 metrics."""

    # Extract aggregate percentages/magnitudes as reported in the raw data
    ES_mean = data['post_rewrite_success'][0]
    ES_std = data['post_rewrite_success'][1]

    EM_mean = data['post_rewrite_diff'][0]
    EM_std = data['post_rewrite_diff'][1]

    PS_mean = data['post_paraphrase_success'][0]
    PS_std = data['post_paraphrase_success'][1]

    PM_mean = data['post_paraphrase_diff'][0]
    PM_std = data['post_paraphrase_diff'][1]

    NS_mean = data['post_neighborhood_success'][0]
    NS_std = data['post_neighborhood_success'][1]

    NM_mean = data['post_neighborhood_diff'][0]
    NM_std = data['post_neighborhood_diff'][1]

    # For S, we must calculate it using the definition (harmonic mean of ES, PS, NS)
    S = calculate_score_s(ES_mean, PS_mean, NS_mean)

    # Fluency and Consistency are direct mean/std from the data
    GE_mean = data['post_ngram_entropy'][0]
    GE_std = data['post_ngram_entropy'][1]

    RS_mean = data['post_reference_score'][0]
    RS_std = data['post_reference_score'][1]

    # 2. Compile the final results dictionary
    results = {
        'S': S,
        'ES_mean': ES_mean, 'ES_std': ES_std,
        'EM_mean': EM_mean, 'EM_std': EM_std,
        'PS_mean': PS_mean, 'PS_std': PS_std,
        'PM_mean': PM_mean, 'PM_std': PM_std,
        'NS_mean': NS_mean, 'NS_std': NS_std,
        'NM_mean': NM_mean, 'NM_std': NM_std,
        'GE_mean': GE_mean, 'GE_std': GE_std,
        'RS_mean': RS_mean, 'RS_std': RS_std,
    }

    return results

def format_output(results: Dict, title: str):
    """Formats the results into the markdown table format, using Unicode characters."""
    output = [f"## Calculated Table 4 Metrics ({title})\n"]

    # Create the metrics table using Unicode characters (± and ↑) to avoid Python backslash issues
    table = [
        "| Metric | Full Name in Table 4 | Result (Mean (± Std Dev)) |",
        "| :--- | :--- | :--- |",
        f"| **S** | Score | **{results['S']:.2f}** (N/A) |",
        f"| **ES** ↑ | Efficacy Score | **{results['ES_mean']:.2f}** (± **{results['ES_std']:.2f}**) |",
        f"| **EM** ↑ | Efficacy Magnitude | **{results['EM_mean']:.2f}** (± **{results['EM_std']:.2f}**) |",
        f"| **PS** ↑ | Paraphrase Score | **{results['PS_mean']:.2f}** (± **{results['PS_std']:.2f}**) |",
        f"| **PM** ↑ | Paraphrase Magnitude | **{results['PM_mean']:.2f}** (± **{results['PM_std']:.2f}**) |",
        f"| **NS** ↑ | Neighborhood Score | **{results['NS_mean']:.2f}** (± **{results['NS_std']:.2f}**) |",
        f"| **NM** ↑ | Neighborhood Magnitude | **{results['NM_mean']:.2f}** (± **{results['NM_std']:.2f}**) |",
        f"| **GE** ↑ | Fluency (n-gram Entropy) | **{results['GE_mean']:.2f}** (± **{results['GE_std']:.2f}**) |",
        f"| **RS** ↑ | Consistency (Reference Score) | **{results['RS_mean']:.2f}** (± **{results['RS_std']:.2f}**) |"
    ]

    output.extend(table)
    return "\n".join(output) + "\n"

# --- Sample Data (Fixed with raw strings for run_dir) ---

# ROME Run 1 (Large N)
data_rome_large = {
    'num_cases': 3831,
    'post_essence_score': (112.84715899554166, 448.00380160329917),
    'post_neighborhood_diff': (4.22, 10.05),
    'post_neighborhood_success': (75.2, 28.92),
    'post_ngram_entropy': (622.31, 21.01),
    'post_paraphrase_diff': (62.0, 35.76),
    'post_paraphrase_success': (96.23, 15.73),
    'post_reference_score': (41.75, 12.49),
    'post_rewrite_diff': (97.73, 8.16),
    'post_rewrite_success': (100.0, 0.0),
    'post_score': (89.05, np.nan),
    'pre_essence_score': (13.282472523775967, 5.097714054357411),
    'pre_neighborhood_diff': (5.12, 8.98),
    'pre_neighborhood_success': (77.98, 27.87),
    'pre_ngram_entropy': (626.61, 12.12),
    'pre_paraphrase_diff': (-4.23, 10.76),
    'pre_paraphrase_success': (23.73, 36.5),
    'pre_reference_score': (31.66, 10.68),
    'pre_rewrite_diff': (-4.96, 13.6),
    'pre_rewrite_success': (21.14, 40.83),
    'pre_score': (29.34, np.nan),
    'run_dir': r'results\ROME\run_000', # FIXED
    'time': (1.7135124301138698, 0.35046412928451093)
}

# ROME Run 2 (Small N)
data_rome_small = {
'num_cases': 52,
 'post_neighborhood_acc': (28.06, 27.05),
 'post_paraphrase_acc': (np.float64(79.19), np.float64(31.43)),
 'post_rewrite_acc': (np.float64(99.45), np.float64(3.92)),
 'post_score': (np.float64(51.44), np.float64(54.69)),
 'pre_neighborhood_acc': (np.float64(28.1), np.float64(27.06)),
 'pre_paraphrase_acc': (np.float64(20.74), np.float64(25.16)),
 'pre_rewrite_acc': (np.float64(20.74), np.float64(25.39)),
 'pre_score': (np.float64(22.72), np.float64(25.56)),
 'run_dir': 'results/ROME/run_001',
 'time': (np.float64(4.263914351279919), np.float64(1.9926132635145974))}


# FT Run 1 (Large N)
data_ft_large = {
    'num_cases': 11758,
    'post_essence_score': (14.011725458048158, 8.822379330307108),
    'post_neighborhood_diff': (3.55, 11.28),
    'post_neighborhood_success': (70.43, 30.58),
    'post_ngram_entropy': (621.92, 44.07),
    'post_paraphrase_diff': (0.36, 14.25),
    'post_paraphrase_success': (48.44, 43.42),
    'post_reference_score': (37.42, 11.13),
    'post_rewrite_diff': (91.58, 19.67),
    'post_rewrite_success': (99.12, 9.32),
    'post_score': (66.77, np.nan),
    'pre_essence_score': (12.473446037809728, 4.27769391401394),
    'pre_neighborhood_diff': (5.11, 8.98),
    'pre_neighborhood_success': (78.13, 28.14),
    'pre_ngram_entropy': (626.73, 11.65),
    'pre_paraphrase_diff': (-4.07, 10.21),
    'pre_paraphrase_success': (24.63, 37.18),
    'pre_reference_score': (31.8, 10.62),
    'pre_rewrite_diff': (-4.93, 13.63),
    'pre_rewrite_success': (22.04, 41.45),
    'pre_score': (30.37, np.nan),
    'run_dir': r'results\FT\run_000', # FIXED
    'time': (1.335606181716854, 0.25900048667326875)
}

# FT Run 2 (Small N)
data_ft_small = {
    'num_cases': 100,
    'post_essence_score': (13.261668682098389, 2.0294880867004395),
    'post_neighborhood_diff': (-2.04, 7.37),
    'post_neighborhood_success': (47.0, 43.09),
    'post_ngram_entropy': (528.78, 50.53),
    'post_paraphrase_diff': (-0.67, 4.24),
    'post_paraphrase_success': (44.5, 46.85),
    'post_reference_score': (6.51, 6.05),
    'post_rewrite_diff': (46.82, 42.69),
    'post_rewrite_success': (96.0, 19.6),
    'post_score': (55.39, np.nan),
    'pre_essence_score': (13.125304222106934, 2.111809730529785),
    'pre_neighborhood_diff': (0.36, 3.22),
    'pre_neighborhood_success': (58.1, 45.53),
    'pre_ngram_entropy': (540.88, 18.84),
    'pre_paraphrase_diff': (-0.42, 3.46),
    'pre_paraphrase_success': (41.5, 45.86),
    'pre_reference_score': (5.61, 3.79),
    'pre_rewrite_diff': (-0.27, 3.72),
    'pre_rewrite_success': (39.0, 48.77),
    'pre_score': (44.81, np.nan),
    'run_dir': r'results\FT\run_004', # FIXED
    'time': (1.2826479744911194, 0.3057939243689511)
}


# --- Execution and Output ---

if __name__ == '__main__':
    print("--- Starting Metric Calculation for Provided Data ---")

    # Process ROME Run 1
    results_rome_large = process_results(data_rome_large)
    print(format_output(results_rome_large, "ROME (Run 1, N=3831)"))

    # Process ROME Run 2
    results_rome_small = process_results(data_rome_small)
    print(format_output(results_rome_small, "ROME (Run 2, N=100)"))

    # Process FT Run 1
    results_ft_large = process_results(data_ft_large)
    print(format_output(results_ft_large, "FT (Run 1, N=11758)"))

    # Process FT Run 2
    results_ft_small = process_results(data_ft_small)
    print(format_output(results_ft_small, "FT (Run 2, N=100)"))

    print("--- Metric Calculation Complete ---")

--- Starting Metric Calculation for Provided Data ---
## Calculated Table 4 Metrics (ROME (Run 1, N=3831))

| Metric | Full Name in Table 4 | Result (Mean (± Std Dev)) |
| :--- | :--- | :--- |
| **S** | Score | **89.05** (N/A) |
| **ES** ↑ | Efficacy Score | **100.00** (± **0.00**) |
| **EM** ↑ | Efficacy Magnitude | **97.73** (± **8.16**) |
| **PS** ↑ | Paraphrase Score | **96.23** (± **15.73**) |
| **PM** ↑ | Paraphrase Magnitude | **62.00** (± **35.76**) |
| **NS** ↑ | Neighborhood Score | **75.20** (± **28.92**) |
| **NM** ↑ | Neighborhood Magnitude | **4.22** (± **10.05**) |
| **GE** ↑ | Fluency (n-gram Entropy) | **622.31** (± **21.01**) |
| **RS** ↑ | Consistency (Reference Score) | **41.75** (± **12.49**) |



KeyError: 'post_rewrite_success'

In [15]:
import numpy as np
import pandas as pd

# ------------------------
# Your experimental results
# ------------------------
Rome = {
 'num_cases': 52,
 'post_neighborhood_acc': (28.06, 27.05),
 'post_paraphrase_acc': (np.float64(79.19), np.float64(31.43)),
 'post_rewrite_acc': (np.float64(99.45), np.float64(3.92)),
 'post_score': (np.float64(51.44), np.float64(54.69)),
 'pre_neighborhood_acc': (np.float64(28.1), np.float64(27.06)),
 'pre_paraphrase_acc': (np.float64(20.74), np.float64(25.16)),
 'pre_rewrite_acc': (np.float64(20.74), np.float64(25.39)),
 'pre_score': (np.float64(22.72), np.float64(25.56)),
 'run_dir': 'results/ROME/run_001',
 'time': (np.float64(4.2639), np.float64(1.9926))
}

FT = {
 'num_cases': 722,
 'post_neighborhood_acc': (np.float64(23.3), np.float64(23.78)),
 'post_paraphrase_acc': (np.float64(32.97), np.float64(33.54)),
 'post_rewrite_acc': (np.float64(61.91), np.float64(36.26)),
 'post_score': (np.float64(33.56), np.float64(np.nan)),
 'pre_neighborhood_acc': (np.float64(24.47), np.float64(24.01)),
 'pre_paraphrase_acc': (np.float64(24.16), np.float64(27.54)),
 'pre_rewrite_acc': (np.float64(24.8), np.float64(28.01)),
 'pre_score': (np.float64(24.47), np.float64(np.nan)),
 'run_dir': 'results/FT/run_000',
 'time': (np.float64(2.0354), np.float64(0.2223))
}

# ------------------------
# Paper values (Table 1: zsRE Editing Results on GPT-2 XL)
# Source: Meng et al., "Locating and Editing Factual Associations in GPT" (Table 1)
# ------------------------
paper_vals = {
    'ROME': {
        'Efficacy':  (99.8, 0.0),
        'Paraphrase': (88.1, 0.5),
        'Specificity': (24.2, 0.5)
    },
    'FT': {
        'Efficacy':  (99.6, 0.1),
        'Paraphrase': (82.1, 0.6),
        'Specificity': (23.2, 0.5)
    }
}

# ------------------------
# Helper function to extract (mean, std) pairs from your data
# ------------------------
def extract_metrics(result_dict):
    def safe_tuple(x):
        try:
            mean_val = float(x[0])
            std_val = float(x[1]) if not np.isnan(x[1]) else 0.0
        except Exception:
            mean_val, std_val = np.nan, np.nan
        return mean_val, std_val

    return {
        'Efficacy': safe_tuple(result_dict.get('post_rewrite_acc', (np.nan, np.nan))),
        'Paraphrase': safe_tuple(result_dict.get('post_paraphrase_acc', (np.nan, np.nan))),
        'Specificity': safe_tuple(result_dict.get('post_neighborhood_acc', (np.nan, np.nan)))
    }

your_rome = extract_metrics(Rome)
your_ft = extract_metrics(FT)

# ------------------------
# Build comparison table
# ------------------------
rows = []
for editor_name, your_vals in [('ROME', your_rome), ('FT', your_ft)]:
    for metric in ['Efficacy', 'Paraphrase', 'Specificity']:
        paper_mean, paper_std = paper_vals.get(editor_name, {}).get(metric, (np.nan, np.nan))
        your_mean, your_std = your_vals.get(metric, (np.nan, np.nan))
        paper_str = f"{paper_mean:.1f} (±{paper_std:.1f})"
        your_str = f"{your_mean:.2f} (±{your_std:.2f})" if not np.isnan(your_mean) else "–"
        diff = your_mean - paper_mean if not np.isnan(your_mean) else np.nan
        diff_str = f"{diff:+.2f}" if not np.isnan(diff) else "–"
        rows.append({
            'Editor': editor_name,
            'Metric': metric,
            'Paper (zsRE, GPT-2 XL)': paper_str,
            'Your Result': your_str,
            'Δ (Yours – Paper)': diff_str
        })

df = pd.DataFrame(rows)

# ------------------------
# Pretty print
# ------------------------
print("\nComparison with paper Table 1 (zsRE Editing Results on GPT-2 XL) — formatted as mean (± std)\n")
print(df.to_string(index=False))



Comparison with paper Table 1 (zsRE Editing Results on GPT-2 XL) — formatted as mean (± std)

Editor      Metric Paper (zsRE, GPT-2 XL)    Your Result Δ (Yours – Paper)
  ROME    Efficacy            99.8 (±0.0)  99.45 (±3.92)             -0.35
  ROME  Paraphrase            88.1 (±0.5) 79.19 (±31.43)             -8.91
  ROME Specificity            24.2 (±0.5) 28.06 (±27.05)             +3.86
    FT    Efficacy            99.6 (±0.1) 61.91 (±36.26)            -37.69
    FT  Paraphrase            82.1 (±0.6) 32.97 (±33.54)            -49.13
    FT Specificity            23.2 (±0.5) 23.30 (±23.78)             +0.10
